### **<mark>GROUP BY AND HAVING</mark>**

Bởi vì SQL Server xử lý mệnh đề HAVING sau mệnh đề GROUP BY, bạn không thể tham chiếu đến hàm tổng hợp được chỉ định trong danh sách chọn bằng cách sử dụng bí danh cột (alias). Truy vấn sau sẽ không thành công:

> <span style="color: #0000ff;">SELECT</span>
> 
>     column\_name1,
> 
>     column\_name2,
> 
>     aggregate\_function (column\_name3) column\_alias
> 
> <span style="color: #0000ff;">FROM</span>
> 
>     table\_name
> 
> <span style="color: #0000ff;">GROUP&nbsp;BY</span>
> 
>     column\_name1,
> 
>     column\_name2
> 
> <span style="color: #0000ff;">HAVING</span>
> 
>     column\_alias <span style="color: #000000;">&gt;</span> <span style="color: #0000ff;">value</span>

Lệnh đúng:

> <span style="color: #0000ff;">SELECT</span>
> 
>     column\_name1,
> 
>     column\_name2,
> 
>     aggregate\_function (column\_name3) alias
> 
> <span style="color: #0000ff;">FROM</span>
> 
>     table\_name
> 
> <span style="color: #0000ff;">GROUP&nbsp;BY</span>
> 
>     column\_name1,
> 
>     column\_name2
> 
> <span style="color: #0000ff;">HAVING</span>
> 
>     aggregate\_function (column\_name3) <span style="color: #000000;">&gt;</span> <span style="color: #0000ff;">value</span>

VD1: Câu lệnh sau sử dụng hàm COUNT () để trả về số lượng sản phẩm có giá lớn hơn 500:

In [ ]:
SELECT
    COUNT(*) product_count
FROM
    production.products
WHERE
    list_price > 500

VD2: Truy vấn sau trả về số lượng khách hàng ở mỗi thành phố

In [ ]:
SELECT
    city,
    COUNT (customer_id) customer_count
FROM
    sales.customers
GROUP BY
    city
ORDER BY
    city

VD3: Câu lệnh sau sử dụng hàm SUM () để tính tổng lượng hàng theo id sản phẩm trong tất cả các kho:

In [ ]:
SELECT 
    product_id, 
    SUM(quantity) stock_count
FROM 
    production.stocks
GROUP BY
    product_id
ORDER BY 
    stock_count DESC

VD4: Câu lệnh sau đây tìm các danh mục sản phẩm có giá niêm yết trung bình từ 500 đến 1.000:

In [ ]:
SELECT
    category_id,
    AVG (list_price) avg_list_price
FROM
    production.products
GROUP BY
    category_id
HAVING
    AVG (list_price) BETWEEN 500 AND 1000

VD5: Câu lệnh sau đây tìm các đơn hàng bán hàng có tổng giá trị lớn hơn 10.000:

In [ ]:
SELECT
    order_id,
    SUM(list_price) AS total
FROM
    sales.order_items
GROUP BY
    order_id
HAVING
    SUM (list_price) > 10000
ORDER BY
    total

VD6: Câu lệnh sau đây trước tiên tìm giá niêm yết tối đa và tối thiểu trong mỗi danh mục sản phẩm. Sau đó, nó lọc ra danh mục có giá niêm yết tối đa lớn hơn 4.000 hoặc giá niêm yết tối thiểu nhỏ hơn 500

In [ ]:
SELECT
    category_id,
    MAX (list_price) max_list_price,
    MIN (list_price) min_list_price
FROM
    production.products
GROUP BY
    category_id
HAVING
    MAX (list_price) > 4000 OR MIN (list_price) < 500

### **<mark>JOINS</mark>**

In [ ]:
CREATE SCHEMA hr
GO

CREATE SCHEMA pm;
GO

CREATE TABLE hr.candidates(
    id INT PRIMARY KEY IDENTITY,
    fullname VARCHAR(100) NOT NULL
);

CREATE TABLE hr.employees(
    id INT PRIMARY KEY IDENTITY,
    fullname VARCHAR(100) NOT NULL
);

CREATE TABLE pm.projects(
    id INT PRIMARY KEY IDENTITY,
    title VARCHAR(255) NOT NULL
);

CREATE TABLE pm.members(
    id INT PRIMARY KEY IDENTITY,
    name VARCHAR(120) NOT NULL,
    project_id INT,
    FOREIGN KEY (project_id) 
        REFERENCES pm.projects(id)
);

INSERT INTO 
    hr.candidates(fullname)
VALUES
    ('John Doe'),
    ('Lily Bush'),
    ('Peter Drucker'),
    ('Jane Doe');


INSERT INTO 
    hr.employees(fullname)
VALUES
    ('John Doe'),
    ('Jane Doe'),
    ('Michael Scott'),
    ('Jack Sparrow');


INSERT INTO 
    pm.projects(title)
VALUES
    ('New CRM for Project Sales'),
    ('ERP Implementation'),
    ('Develop Mobile Sales Platform');


INSERT INTO
    pm.members(name, project_id)
VALUES
    ('John Doe', 1),
    ('Lily Bush', 1),
    ('Jane Doe', 2),
    ('Jack Daniel', null);


**1\. Inner Join:** INNER JOIN tạo ra một tập dữ liệu bao gồm các hàng từ bảng bên trái, khớp với các hàng từ bảng bên phải.

**Cú pháp:**

> <span style="color: #0000ff;">SELECT</span>
> 
>     select\_list
> 
> <span style="color: #0000ff;">FROM</span>
> 
>     T1
> 
> (INNER) <span style="color: #0000ff;">JOIN</span> T2 <span style="color: #0000ff;">ON</span> join\_predicate

In [ ]:
SELECT
    product_name,
    category_name,
    brand_name,
    list_price
FROM
    production.products p
INNER JOIN production.categories c 
    ON c.category_id = p.category_id
INNER JOIN production.brands b 
    ON b.brand_id = p.brand_id
ORDER BY
    product_name DESC

**2\. Left Join:** LEFT JOIN trả về tất cả các hàng từ bảng bên trái và các hàng phù hợp từ bảng bên phải. Nếu không tìm thấy hàng phù hợp nào trong bảng bên phải, thì NULL được sử dụng

**Cú pháp:**

> <span style="color: #0000ff;">SELECT</span>
> 
>     select\_list
> 
> <span style="color: #0000ff;">FROM</span>
> 
>     T1
> 
> <span style="color: #0000ff;">LEFT&nbsp;JOIN</span> T2 
> 
>     <span style="color: #0000ff;">ON</span> join\_predicate

In [ ]:
SELECT
    product_name,
    order_id
FROM
    production.products p
LEFT JOIN sales.order_items o 
    ON o.product_id = p.product_id
ORDER BY
    order_id

**3\. Right Join:** RIGHT JOIN trả về tất cả các hàng từ bảng bên phải và các hàng phù hợp từ bảng bên trái. Nếu không tìm thấy hàng phù hợp nào trong bảng bên phải, thì NULL được sử dụng

**Cú pháp:**

> <span style="color: #0000ff;">SELECT</span> 
> 
>     select\_list
> 
> <span style="color: #0000ff;">FROM</span> 
> 
>     T1
> 
> <span style="color: #0000ff;">RIGHT&nbsp;JOIN</span> T2 
> 
>     <span style="color: #0000ff;">ON</span> join\_predicate

In [ ]:
SELECT
    product_name,
    order_id
FROM
    sales.order_items o
RIGHT JOIN production.products p 
    ON o.product_id = p.product_id
ORDER BY
    order_id

**4\. Full Outer Join:** FULL OUTER JOIN trả về tập kết quả bao gồm các hàng từ cả bảng trái và phải. Khi không có hàng phù hợp nào tồn tại cho hàng trong bảng bên trái, các cột của bảng bên phải sẽ chứa NULL. Tương tự như vậy, khi không có hàng phù hợp nào tồn tại cho hàng trong bảng bên phải, cột của bảng bên trái sẽ chứa NULL.

**Cú pháp:**

> <span style="color: #0000ff;">SELECT</span> 
> 
>     select\_list
> 
> <span style="color: #0000ff;">FROM</span> 
> 
>     T1
> 
> FULL (<span style="color: #0000ff;">OUTER</span>) <span style="color: #0000ff;">JOIN</span> T2 
> 
>     <span style="color: #0000ff;">ON</span> join\_predicate

In [ ]:
SELECT 
    m.name member, 
    p.title project
FROM 
    pm.members m
FULL OUTER JOIN pm.projects p 
    ON p.id = m.project_id

In [ ]:
SELECT 
    m.name member, 
    p.title project
FROM 
    pm.members m
FULL OUTER JOIN pm.projects p 
    ON p.id = m.project_id
WHERE
    m.id IS NULL OR
    P.id IS NULL

**5\. Cross Join:** CROSS JOIN đã nối mọi hàng từ bảng đầu tiên với mọi hàng từ bảng thứ hai. Nói cách khác, phép nối chéo trả về tích Descartes của các hàng từ cả hai bảng. Không giống như INNER JOIN hoặc LEFT JOIN, phép nối chéo không thiết lập mối quan hệ giữa các bảng đã tham gia.

Cú pháp: 

> <span style="color: #0000ff;">SELECT</span>
> 
>     select\_list
> 
> <span style="color: #0000ff;">FROM</span>
> 
>     T1
> 
> <span style="color: #0000ff;">CROSS</span> <span style="color: #0000ff;">JOIN</span> T2;

In [ ]:
SELECT
    product_id,
    product_name,
    store_id,
    0 AS quantity
FROM
    production.products
CROSS JOIN sales.stores
ORDER BY
    product_name,
    store_id;

**6\. Self Join:** SELF JOIN cho phép bạn tham gia một bảng với chính nó. Nó giúp truy vấn dữ liệu phân cấp (hierarchical data) hoặc so sánh các hàng trong cùng một bảng. SELF JOIN sử dụng mệnh đề INNER JOIN hoặc LEFT JOIN. Bởi vì truy vấn sử dụng liên kết tự tham chiếu đến cùng một bảng, bí danh bảng được sử dụng để gán các tên khác nhau cho cùng một bảng trong truy vấn.

**Cú pháp:** 

> <span style="color: #0000ff;">SELECT</span>
> 
>     select\_list
> 
> <span style="color: #0000ff;">FROM</span>
> 
>     T t1
> 
> \[INNER | LEFT\]  <span style="color: #0000ff;">JOIN</span> T t2 <span style="color: #0000ff;">ON</span>
> 
>     join\_predicate;

In [ ]:
SELECT
    e.first_name + ' ' + e.last_name employee,
    m.first_name + ' ' + m.last_name manager
FROM
    sales.staffs e
INNER JOIN sales.staffs m ON m.staff_id = e.manager_id
ORDER BY
    manager;

In [ ]:
-- Câu lệnh sau đây sử dụng phép nối tự để tìm khách hàng ở trong cùng một thành phố.
SELECT
    c1.city,
    c1.first_name + ' ' + c1.last_name customer_1,
    c2.first_name + ' ' + c2.last_name customer_2
FROM
    sales.customers c1
INNER JOIN sales.customers c2 ON c1.customer_id > c2.customer_id
AND c1.city = c2.city
ORDER BY
    city,
    customer_1,
    customer_2;

In [ ]:
SELECT
    c1.city,
    c1.first_name + ' ' + c1.last_name customer_1,
    c2.first_name + ' ' + c2.last_name customer_2
FROM
    sales.customers c1
INNER JOIN sales.customers c2 ON c1.customer_id != c2.customer_id
AND c1.city = c2.city
ORDER BY
    city,
    customer_1,
    customer_2;

### **Conditions in <span style="color: #0000ff;">ON</span> vs. <span style="color: #0000ff;">WHERE</span> clause**

In [ ]:
SELECT
    product_name,
    order_id
FROM
    production.products p
LEFT JOIN sales.order_items o 
   ON o.product_id = p.product_id
WHERE order_id = 100
ORDER BY
    order_id;

In [ ]:
SELECT
    p.product_id,
    product_name,
    order_id
FROM
    production.products p
LEFT JOIN sales.order_items o 
        ON o.product_id = p.product_id AND 
        o.order_id = 100
ORDER BY
    order_id DESC;

### **Thứ tự thực thi lệnh trên SQL Server (Order of Execution)**

> <span style="color: #0000ff;">SELECT&nbsp;DISTINCT</span> | <span style="color: #0000ff;">TOP</span> column, AGG\_FUNC(column\_or\_expression), …
> 
> <span style="color: #0000ff;">FROM</span> mytable
> 
> <span style="color: #0000ff;">JOIN</span> another\_table
> 
>     <span style="color: #0000ff;">ON</span> mytable.column <span style="color: #000000;">=</span> another\_table.column
> 
> <span style="color: #0000ff;">WHERE</span> constraint\_expression
> 
> <span style="color: #0000ff;">GROUP&nbsp;BY</span> column
> 
> <span style="color: #0000ff;">HAVING</span> constraint\_expression
> 
> <span style="color: #0000ff;">ORDER&nbsp;BY</span> column <span style="color: #0000ff;">ASC</span><span style="color: #000000;">/</span><span style="color: #0000ff;">DESC</span>

1\. FROM và JOIN(s)

2\. WHERE

3\. GROUP BY

4\. HAVING

5\. SELECT

6\. DISTINCT

7\. ORDER BY

8\. TOP

In [ ]:
SELECT
    column_name1,
    column_name2,
    aggregate_function (column_name3) column_alias
FROM
    table_name
GROUP BY
    column_name1,
    column_name2
HAVING
    column_alias > value;

In [ ]:
SELECT
    order_id,
    SUM(list_price) AS total
FROM
    sales.order_items
GROUP BY
    order_id
HAVING
    total > 10000
ORDER BY
    total

In [ ]:
SELECT
    column_name1,
    column_name2,
    aggregate_function (column_name3) alias
FROM
    table_name
GROUP BY
    column_name1,
    column_name2
HAVING
    aggregate_function (column_name3) > value

In [ ]:
SELECT
    order_id,
    SUM(list_price) AS total
FROM
    sales.order_items
GROUP BY
    order_id
HAVING
    SUM(list_price) > 10000
ORDER BY
    total